# Pull request scraper

## GitHub credentials
A private access token is necessary to make use of less restrictive API limits.

In [1]:
from github import RateLimitExceededException, Github

# Providing access token
access_token = ""
g = Github(login_or_token=access_token)

# Confirm your login is successful
user = g.get_user()
print(f"Authenticated as: {user.login}")

Authenticated as: AbelvdTil


## Files

In [2]:
import os
 
STEP4_TFCOMMITS = os.path.join("data", "previous-study", "step4-tf-commits.json") 
COMMIT_LABELS = os.path.join("data", "process-labeled-commits", "full-commit-labels.json") 

STEP5_TF_REPOS_WITH_PR = os.path.join("data", "pullrequest-scraping", "step5-tf-repos-with-pr.json")
STEP6_TF_REPOS_COMMITS = os.path.join("data", "pullrequest-scraping", "step6-tf-repos-commits.json")
STEP6A_TF_REPOS_RELEVANT_COMMITS = os.path.join("data", "pullrequest-scraping", "step6a-tf-repos-relevant-commits.json")
STEP7_TF_REPOS_WITH_TF_PR = os.path.join("data", "pullrequest-scraping", "step7-tf-repos-with-tf-pr.json")
STEP8_TF_KEYWORD_PR = os.path.join("data", "pullrequest-scraping", "step8-tf-keyword-pr.json")
STEP9_TF_PR_DATASET = os.path.join("data", "pullrequest-scraping", "step9-tf-pr-dataset.json")

## File seperator and combinator

Files over 100MB are not stored on GitHub, therefore we need to seperate large files into smaller ones.
Any step5 and step7 files can be split into multiple smaller ones and be combined together.

The repositories are split on pull request level. The first pull request is stored in part 1, the 2nd in part 2 etc. This will make sure that it is fairly equally distributed.

Each pull request is accomodated with the url of the repository. Therefore it is possible to reconstruct the original file again.

In [3]:
import json
import math

FILE_TO_SEPERATE = STEP5_TF_REPOS_WITH_PR

nr_parts = 10

### File seperator

In [28]:
parts_data = []

for i in range(nr_parts):
    parts_data.append([])

file = open(FILE_TO_SEPERATE)
seperator_data = json.load(file)

count = 0
for repository in seperator_data:
    for pull_request in repository["pull_requests"]:
        part = (count % nr_parts)
        count += 1
        parts_data[part].append({"repo_url": repository["url"], "pull_request": pull_request})

for i in range(nr_parts):
    with open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(i+1) + ".json", "w") as outfile:
        json.dump(parts_data[i], outfile)

39062


### File combinator

In [4]:
url_dict = {}
combinator_data = []
index_count = 0
for part in range(nr_parts):
    part_file = open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(part+1) + ".json", "r")
    part_data = json.load(part_file)

    for nugget in part_data:
        # find repo using url
        index = url_dict.get(nugget["repo_url"], None)

        if (index == None):
            repo = {"url": nugget["repo_url"], "pull_requests": []}
            combinator_data.append(repo)
            
            url_dict[nugget["repo_url"]] = index_count
            index_count += 1
        else:
            repo = combinator_data[index]
            
        repo["pull_requests"].append(nugget["pull_request"])

with open(FILE_TO_SEPERATE, "w") as outfile:
    json.dump(combinator_data, outfile) 

## STEP 5: Pull request scraping script

For each repository, get all pull request data. This includes PR Title, description, (review) comments and commit hashes. Exclude any repositories that do not have pull requests.

In [8]:
# SETTINGS
check_limit_every_x_calls = 5
api_limit_buffer = 10
api_calls_per_debug = 500

# INITIALIZATION

import json
import datetime
import time

# INITIALIZATION
terraform_output = open(STEP4_TFCOMMITS)
step4_output = json.load(terraform_output)

# Retrieve data from previous run
try:
    previous_run = open(STEP5_TF_REPOS_WITH_PR)
    repoData_dict = json.load(previous_run)
except FileNotFoundError as e:
    repoData_dict = []

iteration = 0
calls_till_next_debug = 0
calls_till_limit_checkup = 0

# Check for api limits, also periodically calls print debug.
def CheckForApiLimit():
    global calls_till_limit_checkup
    global calls_till_next_debug
    global api_calls_per_debug
    global api_limit_buffer

    # check for limit
    if (calls_till_limit_checkup == 0):
        core_limit = g.get_rate_limit().core

        # sleep when exceeded api core limit
        if (core_limit.remaining <= api_limit_buffer):
            time_to_sleep = core_limit.raw_data['reset'] - time.time() + 1
            print("Rate limit exceeded, sleeping for", time_to_sleep, "seconds.", "Actual remaining calls", core_limit.remaining)
            time.sleep(time_to_sleep)

        calls_till_limit_checkup = check_limit_every_x_calls
    
    calls_till_limit_checkup -= 1

    # check for debug
    if (calls_till_next_debug == 0):
        PrintDebug()
        calls_till_next_debug = api_calls_per_debug

    calls_till_next_debug -= 1

# Prints debug message
def PrintDebug():
    global iteration
    global repo_url

    print(datetime.datetime.now().strftime("%H:%M:%S"), ":", 
              "current iteration:", iteration, 
              "url:", repo_url)

# Pull request scraping script
for rp in step4_output["repositories"]:
    try:
        iteration += 1
            
        repo_url = rp["name"]

        # skip already scraped repositories
        if any(d["url"] == repo_url for d in repoData_dict):
            continue

        # Get the repo object from the url
        split_list = repo_url.split("/")
        actual_url = (split_list[3]+ '/' + split_list[4]).split('.git')[0]
        repo = g.get_repo(actual_url)
        
        # Get required info for pull requests
        pull_requests_dict = []
        pull_requests = repo.get_pulls(state="closed")

        if pull_requests.totalCount > 0:
            for pr in pull_requests:
    
                # retrieve all review comments, not required if there are none.
                comments = []
            
                for review in pr.get_reviews():
                    if (review.body.strip() != ""):
                        comments.append(review.body)
                CheckForApiLimit()
               
                for comment in pr.get_issue_comments():
                    if (comment.body.strip() != ""):
                        comments.append(comment.body)
                CheckForApiLimit()
    
                # retrieve all connected commits.
                commits = []
                for commit in pr.get_commits():
                    commits.append(commit.sha)
                CheckForApiLimit()
    
                pull_requests_dict.append({"url": pr.html_url, "title": pr.title, "body": pr.body, "comments": comments, "commits": commits})
            
            CheckForApiLimit()        
            repoData_dict.append({"url": repo_url, "pull_requests": pull_requests_dict});
            
            with open(STEP5_TF_REPOS_WITH_PR, "w") as outfile:
                json.dump(repoData_dict, outfile)
    except Exception as e:
        print("exception:", e)

tkhoa2711/terraform-digitalocean


KeyboardInterrupt: 

In [3]:
# SETTINGS
check_limit_every_x_calls = 5
api_limit_buffer = 10
api_calls_per_debug = 500

outputfile = os.path.join("data", "pullrequest-scraping", "missing-comments.json")

# INITIALIZATION

import json
import datetime
import time

repoData_dict = []

# INITIALIZATION
terraform_output = open(STEP5_TF_REPOS_WITH_PR)
step4_output = json.load(terraform_output)

# Retrieve data from previous run
try:
    previous_run = open(outputfile)
    repoData_dict = json.load(previous_run)
except FileNotFoundError as e:
    repoData_dict = []

iteration = 0
calls_till_next_debug = 0
calls_till_limit_checkup = 0

# Check for api limits, also periodically calls print debug.
def CheckForApiLimit():
    global calls_till_limit_checkup
    global calls_till_next_debug
    global api_calls_per_debug
    global api_limit_buffer

    # check for limit
    if (calls_till_limit_checkup == 0):
        core_limit = g.get_rate_limit().core

        # sleep when exceeded api core limit
        if (core_limit.remaining <= api_limit_buffer):
            time_to_sleep = core_limit.raw_data['reset'] - time.time() + 1
            print("Rate limit exceeded, sleeping for", time_to_sleep, "seconds.", "Actual remaining calls", core_limit.remaining)
            time.sleep(time_to_sleep)

        calls_till_limit_checkup = check_limit_every_x_calls
    
    calls_till_limit_checkup -= 1

    # check for debug
    if (calls_till_next_debug == 0):
        PrintDebug()
        calls_till_next_debug = api_calls_per_debug

    calls_till_next_debug -= 1

# Prints debug message
def PrintDebug():
    global iteration
    global repo_url

    print(datetime.datetime.now().strftime("%H:%M:%S"), ":", 
              "current iteration:", iteration, 
              "url:", repo_url)

# Pull request scraping script
for rp in step4_output:
    try:
        iteration += 1
            
        repo_url = rp["url"]

        # skip already scraped repositories
        if any(d["url"] == repo_url for d in repoData_dict):
            continue

        # Get the repo object from the url
        split_list = repo_url.split("/")
        actual_url = (split_list[3]+ '/' + split_list[4]).split('.git')[0]
        repo = g.get_repo(actual_url)
        
        # Get required info for pull requests
        pull_requests_dict = []
        pull_requests = repo.get_pulls(state="closed")

        if pull_requests.totalCount > 0:
            print(repo_url)
            for pr in pull_requests:
    
                # retrieve all review comments, not required if there are none.
                comments = []
                if (pr.review_comments == 0):
                    for review in pr.get_reviews():
                        if (review.body.strip() != ""):
                            comments.append(review.body)
                    CheckForApiLimit()

                if (pr.comments == 0):
                    for comment in pr.get_issue_comments():
                        if (comment.body.strip() != ""):
                            comments.append(comment.body)
                    CheckForApiLimit()

                if (len(comments) > 0):
                    pull_requests_dict.append({"url": pr.html_url, "comments": comments})
            
            CheckForApiLimit()     
            
            repoData_dict.append({"url": repo_url, "pull_requests": pull_requests_dict});

            with open(outputfile , "w") as outfile:
                json.dump(repoData_dict, outfile)
    except Exception as e:
        print("exception:", e)

https://github.com/GaloyMoney/charts.git
06:19:20 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
06:27:05 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
06:35:01 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
06:42:47 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
06:50:32 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
06:58:03 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
07:05:42 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
Rate limit exceeded, sleeping for 519.1453416347504 seconds. Actual remaining calls 5
07:22:02 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
07:29:41 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
07:37:34 : current iteration: 143 url: https://github.com/GaloyMoney/charts.git
07:45:21 : current iteration: 143 url: https://github.com/GaloyMoney/char

Following Github server redirection from /repos/GlueOps/terraform-gcp-organization-bootstrap to /repositories/479564327


https://github.com/GlueOps/terraform-gcp-organization-bootstrap.git
https://github.com/NDPH-ARTS/mts-trial-deployment-config.git
https://github.com/picatz/terraform-google-nomad.git
https://github.com/mozilla-platform-ops/relops_infra_as_code.git
11:38:54 : current iteration: 164 url: https://github.com/mozilla-platform-ops/relops_infra_as_code.git
https://github.com/y-miyazaki/terraform-aws-base.git
https://github.com/terraform-google-modules/terraform-example-foundation.git
11:48:40 : current iteration: 166 url: https://github.com/terraform-google-modules/terraform-example-foundation.git
11:59:18 : current iteration: 166 url: https://github.com/terraform-google-modules/terraform-example-foundation.git
https://github.com/ministryofjustice/opg-infra-costs-action.git
https://github.com/UKHomeOffice/dq-tf-infra.git
https://github.com/dhoppeIT/terraform-tfe-organization.git
12:07:56 : current iteration: 169 url: https://github.com/dhoppeIT/terraform-tfe-organization.git
https://github.com

Following Github server redirection from /repos/rajesh-nitc/gcp-foundation to /repositories/367584349


https://github.com/rajesh-nitc/gcp-foundation.git
14:22:01 : current iteration: 193 url: https://github.com/rajesh-nitc/gcp-foundation.git
https://github.com/datarootsio/terraform-module-azure-datalake.git
14:30:57 : current iteration: 194 url: https://github.com/datarootsio/terraform-module-azure-datalake.git
https://github.com/mintak21/terraform-old.git
https://github.com/ministryofjustice/hmpps-env-configs.git
14:39:09 : current iteration: 196 url: https://github.com/ministryofjustice/hmpps-env-configs.git
14:47:18 : current iteration: 196 url: https://github.com/ministryofjustice/hmpps-env-configs.git
14:55:34 : current iteration: 196 url: https://github.com/ministryofjustice/hmpps-env-configs.git
15:03:49 : current iteration: 196 url: https://github.com/ministryofjustice/hmpps-env-configs.git
15:12:04 : current iteration: 196 url: https://github.com/ministryofjustice/hmpps-env-configs.git
Rate limit exceeded, sleeping for 392.44300961494446 seconds. Actual remaining calls 9
15:26:

...

https://github.com/samq-ghdemo/terragoat.git
https://github.com/mantvydasb/Red-Team-Infrastructure-Automation.git
https://github.com/liamstevens/gcp-pca-prep.git
https://github.com/cob16/aws_static_website.git
https://github.com/DNXLabs/terraform-aws-idp-gsuite.git
https://github.com/saraangelmurphy/terraform-gcp-createProject.git
https://github.com/JamesWoolfenden/terraform-aws-rds.git


Following Github server redirection from /repos/bphanbc/terragoat to /repositories/477081171


https://github.com/bphanbc/terragoat.git
https://github.com/peemblis/hashicat-gcp.git
https://github.com/JamesWoolfenden/terraform-gcp-staticip.git
https://github.com/DNXLabs/terraform-aws-account.git
https://github.com/wsierakowski/woj-infra-tf.git
https://github.com/JamesWoolfenden/terraform-aws-vpc-peering.git
https://github.com/akentosh/ec2-testing.git
https://github.com/greyhats13/avana-aws-infra.git
https://github.com/ecsworkshop2018/expertalk-2018-ecs-workshop.git
https://github.com/exekube/demo-grpc-project.git
https://github.com/JamesWoolfenden/terraform-gcp-statebucket.git
https://github.com/nikkiwritescode/flask-app-terraform-deployment.git
https://github.com/renovate-tests/hackstack-local.git
https://github.com/aiidalab/aiidalab-k8s.git
Rate limit exceeded, sleeping for 633.2105679512024 seconds. Actual remaining calls 2
https://github.com/cider-demo/Imperial-Shuttle.git
https://github.com/MyCryptoHQ/MoonpaySigner.git
https://github.com/goodpen/gke-v.1.0.git
https://github.

Following Github server redirection from /repos/palogitjl/terragoat1 to /repositories/441021321


https://github.com/palogitjl/terragoat1.git
https://github.com/maxjahn/terraform-oci-azure-latency-shootout.git


## STEP 6: Find tf commits for tf repos with pr's 

For remaining repositories from step 5, collect all commits that modify a terraform file.

In [14]:
from pydriller import Repository

import json

step5 = open(STEP5_TF_REPOS_WITH_PR)
tf_repositories = json.load(step5)

terraform_keywords = ['.tf', '.tf.json']

iteration = 0
    
# Pull request scraping script
repo_dic = []
for repository in tf_repositories:
    try:
        if (iteration % 50 == 0):
            print("at iteration", iteration)
            with open(STEP6_TF_REPOS_COMMITS, "w") as outfile:
                json.dump(repo_dic, outfile)

        iteration += 1

        # Get each commit
        commit_dic = []
        for commit in Repository(repository["url"]).traverse_commits():

            modified_terraform = False
            # find if it changes a terraform file
            for file in commit.modified_files:
                if any(key in file.filename for key in terraform_keywords):
                    modified_terraform = True
            
            if modified_terraform:
                commit_dic.append({"hash": commit.hash, 
                                   "url": repository["url"].split(".git")[0] + "/commit/" + commit.hash, 
                                   "date": str(commit.author_date), 
                                   "body": commit.msg})
  
        repo_dic.append({"url":repository["url"], "commits":commit_dic})
        
    except Exception as e:
        print("exception:", e)

with open(STEP6_TF_REPOS_COMMITS, "w") as outfile:
        json.dump(repo_dic, outfile)

at iteration 0
at iteration 50
at iteration 100
at iteration 150
at iteration 200
at iteration 250
at iteration 300
at iteration 350
at iteration 400
at iteration 450
at iteration 500
at iteration 550
at iteration 600


## STEP 6b: exclude unrelated tf commits

In [16]:
import json

step6 = open(STEP6_TF_REPOS_COMMITS)
repository_commits = json.load(step6)

labels_file = open(COMMIT_LABELS)
commit_labels = json.load(labels_file)

for repository in repository_commits:
    commits = []
    for commit in repository["commits"]:
        label = commit_labels.get(commit["hash"], None)
        
        if label is None or "unrelated" not in label:
            commits.append(commit)
    repository["commits"] = commits

with open(STEP6A_TF_REPOS_RELEVANT_COMMITS, "w") as outfile:
        json.dump(repository_commits, outfile)

## STEP 7: filter out pull requests without relevant tf commit

Removes any pull request that does not include a commit from the previous step, for the remaining pull requests, it combines the two datasets into one.

In [25]:
import json

step5 = open(STEP5_TF_REPOS_WITH_PR)
repository_input = json.load(step5)

step6a = open(STEP6A_TF_REPOS_RELEVANT_COMMITS)
commit_input = json.load(step6a)

iteration = 0

output_dict = []

# for each repository
for repository in repository_input:
    # find commits for repo from step 6a
    commit_input_list = next(repo["commits"] for repo in commit_input if repo["url"] == repository["url"])

    pr_dict = []
    # for each pull request
    for pull_request in repository["pull_requests"]:
        commit_dict = []

        # for each commit
        for commit_hash in pull_request["commits"]:
            # Find the exact commit from step 7
            commit_data = next((commit for commit in commit_input_list if commit["hash"] == commit_hash), None)
            if (commit_data is not None):
                commit_dict.append(commit_data)

        pull_request["total_commits"] = len(pull_request["commits"])
        pull_request["commits"] = commit_dict
        
        if (len(commit_dict) > 0):
            pr_dict.append(pull_request)
    
    if (len(pr_dict) > 0):
        output_dict.append({"url": repository["url"], "pull_requests": pr_dict})

with open(STEP7_TF_REPOS_WITH_TF_PR, "w") as outfile:
    json.dump(output_dict, outfile) 

## STEP 8: list all tf pull request with a keyword

In [26]:
cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

step7 = open(STEP7_TF_REPOS_WITH_TF_PR)
repo_input = json.load(step7)

pullrequest_dict_output = []
for repository in repo_input:
    for pr in repository["pull_requests"]:
        
        title   = True if (pr["title"]        is not None and any(key in pr["title"].lower()    for key in cost_keywords)) else False
        body    = True if (pr["body"]         is not None and any(key in pr["body"].lower()     for key in cost_keywords)) else False
        comment = True if (any(comment        is not None and     key in comment.lower()        for key in cost_keywords for comment in pr["comments"])) else False
        commit  = True if (any(commit["body"] is not None and     key in commit["body"].lower() for key in cost_keywords for commit  in pr["commits"]))  else False
            
        reason = (("title " if title else "") + 
                  ("body " if body else "") + 
                  ("comment " if comment else "") + 
                  ("commit " if commit else ""))
        
        if (title or body or comment or commit):
            pullrequest_dict_output.append({"reason": reason.strip(), "pull_request": pr})

with open(STEP8_TF_KEYWORD_PR, "w") as outfile:
    json.dump(pullrequest_dict_output, outfile) 

## STEP 9: Parse to dataset format

In [7]:
import json

step8 = open(STEP8_TF_KEYWORD_PR)
pull_request_reasons = json.load(step8)

pr_output = []
for pull_request_reason in pull_request_reasons:
    pull_request = pull_request_reason["pull_request"]
    commits = []
    for commit in pull_request["commits"]:
        commits.append(commit["hash"])
    pr_output.append(
        {
            "type": "pull_request", 
            "url": pull_request["url"],
            "content": {
                "title": pull_request["title"],
                "body": pull_request["body"],
                "comments": pull_request["comments"],
                "commits": commits
                },
            "codes": []
        })

with open(STEP9_TF_PR_DATASET, "w") as outfile:
    json.dump(pr_output, outfile) 

## STEP 9: results

In [34]:
import json

cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

step4 = open(STEP4_TFCOMMITS)
step4_data = json.load(step4)

step5 = open(STEP5_TF_REPOS_WITH_PR)
step5_data = json.load(step5)

step7 = open(STEP7_TF_REPOS_WITH_TF_PR)
repo_input = json.load(step7)

step8 = open(STEP8_TF_KEYWORD_PR)
pr_reason_input = json.load(step8)


# GENERAL REPOSITORY DATA

print("Total repositories:" , step4_data["no_of_repos"])
print("Of those that exist and have pull request(s):", len(step5_data))
print("Of those that have relevant TF commits:", len(repo_input))

print("")

# GENERAL PULL REQUEST DATA
print("Total TF PR's with a keyword:", len(pr_reason_input))

print("")

print("PR with keyword in:\t", "Only in:")
print("Title:\t\t", len([pr for pr in pr_reason_input if "title" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "title" == pr["reason"]]))
print("Description:\t", len([pr for pr in pr_reason_input if "body" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "body" == pr["reason"]]))
print("Comment:\t", len([pr for pr in pr_reason_input if "comment" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "comment" == pr["reason"]]))
print("commit message*:", len([pr for pr in pr_reason_input if "commit" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "commit" == pr["reason"]]))  

print("")

print("*commits labeled as unrelevant have already been removed from the dataset, while the same is not true for the other locations.")

print("")

# GENERAL COMMIT DATA

print("Total amount of relevant commits in TF PR's:", sum([pr["total_commits"] for repo in repo_input for pr in repo["pull_requests"]]))
print("Of those that modify a TF file:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"]]))
print("Of those that have a keyword:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"] if any(key in commit["body"].lower() for key in cost_keywords)]))

print("")

count = 0
for repo in repo_input:
    for pr in repo["pull_requests"]:
        if (pr["total_commits"] >= 250):
            count += 1
print("Amount of PR's with more than 250 commits (limit):", count)
    


Total repositories: 1278
Of those that exist and have pull request(s): 610
Of those that have relevant TF commits: 469

Total TF PR's with a keyword: 760

PR with keyword in:	 Only in:
Title:		 111 	 30
Description:	 363 	 216
Comment:	 297 	 268
commit message*: 194 	 63

*commits labeled as unrelevant have already been removed from the dataset, while the same is not true for the other locations.

Total amount of relevant commits in TF PR's: 130720
Of those that modify a TF file: 3186
Of those that have a keyword: 203

Amount of PR's with more than 250 commits (limit): 34
